In [ ]:
import os
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By

def encontrar_elemento(driver, metodo, seletor, tempo=10):
    # Aguardar a presença do elemento na página e efetua o retorno
    return WebDriverWait(driver, tempo).until(EC.presence_of_element_located((metodo, seletor)))

def wait_for_alert(driver):
        # Aguarda um alerta estar presente na página e retorna o mesmo
        return WebDriverWait(driver, 10).until(EC.alert_is_present())

# armazenando caminho do arquivo local
caminho = os.getcwd()
caminho_salvar_planilha = os.path.join(caminho, "Processos")

# criando a pasta das planilha de processos, caso não exista
os.makedirs(caminho_salvar_planilha, exist_ok=True)

# concatenando endereço da pagina
arquivo = caminho + "/index.html"

# importando tabela de dados
tabela_dados = pd.read_excel("Processos.xlsx")

# inicializando navegador
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

# Inicializar o Webdriver dentro de um gerenciamento de contexto
with webdriver.Chrome(options=options) as navegador:
    try:
        # armazenando o tempo e horario, para utilizar na criação das subpastas
        timestamp = datetime.now().strftime("%d%m%Y_%H%M%S")
        for linha in tabela_dados.index:

            # abrindo pagina local
            navegador.get(rf"file:///{arquivo}")

            # Interagindo com o dropdown das cidades
            hover_button_state = encontrar_elemento(navegador, By.CLASS_NAME, 'dropdown-menu')
            ActionChains(navegador).move_to_element(hover_button_state).perform()

            cidade = tabela_dados.loc[linha, 'Cidade']
            # selecionando a cidade no dropdown
            navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()

            # Esperando o momento de abertura da nova aba
            WebDriverWait(navegador, 10).until(lambda wnd: len(wnd.window_handles) > 1)

            # retorna lista ordenada contendo os identificadores de todas as abas abertas no navegador
            abas_navegador = navegador.window_handles
            # abas_navegador[-1] => aba mais recente, ou seja, a nova aba que foi aberta
            navegador.switch_to.window(abas_navegador[-1])

            # Preencher campo Nome
            encontrar_elemento(navegador, By.ID, 'nome').send_keys(tabela_dados.loc[linha, "Nome"])
            time.sleep(0.7)

            # Advogado
            encontrar_elemento(navegador, By.ID, 'advogado').send_keys(tabela_dados.loc[linha, "Advogado"])
            time.sleep(0.7)

            # Número do processo
            encontrar_elemento(navegador, By.ID, 'numero').send_keys(tabela_dados.loc[linha, "Processo"])

            # clicando no botão de pesquisar
            WebDriverWait(navegador, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'registerbtn'))).click()

            # realizando tratativa dos popups(alertas)
            alerta_confirmacao = wait_for_alert(navegador)
            time.sleep(2)
            alerta_confirmacao.accept()

            alerta_resultado = wait_for_alert(navegador)
            texto_alerta = alerta_resultado.text

            if 'Processo encontrado com sucesso' in texto_alerta:
                time.sleep(2)
                alerta_resultado.accept()
                tabela_dados.loc[linha, "Status"] = "✅ Processo encontrado"
            else:
                time.sleep(2)
                alerta_resultado.accept()
                tabela_dados.loc[linha, "Status"] = "❌ Processo não encontrado"

    except Exception as e:
        with open("erros.log", "a") as log_file:
            log_file.write(f"Erro na linha {linha}: {str(e)}\n")
    finally:
        # fechando navegador
        time.sleep(4)
        tabela_dados.to_excel(f"{caminho_salvar_planilha}/Processos_status_atualizados_{timestamp}.xlsx", index=False)
        navegador.quit()

In [ ]:
# tabela_dados = pd.read_excel("Processos.xlsx")
display(tabela_dados)
# tabela_dados.to_excel("Processos_status_atualizados.xlsx")